In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

In [4]:
import pandas
df = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/ctbc/master/data/yahoo_movie.xlsx')
df.head()

,content,stars,title,status
0,"不知道耶! 看完整個無感\r\n有種覺得就是女兒""給蕭""害死了老爸\r\n然後...既然可以...",3,古墓奇兵,soso
1,很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。女主角跟爸爸...,5,古墓奇兵,good
2,說這個不好看的話，那我還真不知道，還有什麼片是您可以去看得了。電影好看，但話說羅拉有裝可以撿...,5,古墓奇兵,good
3,父女重逢真的很讓人感動，五顆星。,5,古墓奇兵,good
4,劇情雖然老套，但仍拍出新意，古墓能殺人的方式不就是機關和毒，要求亂七八糟的觀眾，你看喪尸片看多了。,4,古墓奇兵,good


In [5]:
df = df[df['status'].isin(['good', 'bad'])]
df.head()

,content,stars,title,status
1,很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。女主角跟爸爸...,5,古墓奇兵,good
2,說這個不好看的話，那我還真不知道，還有什麼片是您可以去看得了。電影好看，但話說羅拉有裝可以撿...,5,古墓奇兵,good
3,父女重逢真的很讓人感動，五顆星。,5,古墓奇兵,good
4,劇情雖然老套，但仍拍出新意，古墓能殺人的方式不就是機關和毒，要求亂七八糟的觀眾，你看喪尸片看多了。,4,古墓奇兵,good
5,每個演員都長得像路人，劇情解謎上都沒有作一些鋪陳，以致於每個謎題只有主角轉一轉就都破解了，覺...,2,古墓奇兵,bad


In [7]:
import jieba
df['label'] = df['status']
df['cutword'] = df['content'].map(lambda e: ' '.join(jieba.cut(e)) ) 

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/46/b7dzk4mn6g54qzptv608w7d00000gn/T/jieba.cache
Loading model cost 0.826 seconds.
Prefix dict has been built succesfully.


In [8]:
msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]

In [9]:
## 对数据集的标签数据进行编码
train_y = train_df.label
test_y = test_df.label
le = LabelEncoder()
train_y = le.fit_transform(train_y).reshape(-1,1)
test_y = le.transform(test_y).reshape(-1,1)

## 对数据集的标签数据进行one-hot编码
ohe = OneHotEncoder()
train_y = ohe.fit_transform(train_y).toarray()
test_y = ohe.transform(test_y).toarray()

In [10]:
## 使用Tokenizer对词组进行编码
## 当我们创建了一个Tokenizer对象后，使用该对象的fit_on_texts()函数，以空格去识别每个词,
## 可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小。
max_words = 5000
max_len = 600
tok = Tokenizer(num_words=max_words)  ## 使用的最大词语数为5000
tok.fit_on_texts(train_df.cutword)

## 使用word_index属性可以看到每次词对应的编码
## 使用word_counts属性可以看到每个词对应的频数
for ii,iterm in enumerate(tok.word_index.items()):
    if ii < 10:
        print(iterm)
    else:
        break
print("===================")  
for ii,iterm in enumerate(tok.word_counts.items()):
    if ii < 10:
        print(iterm)
    else:
        break

('的', 1)
('，', 2)
('\r', 3)
('看', 4)
('了', 5)
('是', 6)
('我', 7)
('有', 8)
('。', 9)
('劇情', 10)
('很', 231)
('好看', 207)
('的', 1396)
('動作片', 10)
('，', 1347)
('不會', 37)
('浪費', 22)
('錢', 45)
('跟', 94)
('時間', 26)


In [11]:
## 对每个词编码之后，每句新闻中的每个词就可以用对应的编码表示，即每条新闻可以转变成一个向量了：
train_seq = tok.texts_to_sequences(train_df.cutword)
test_seq = tok.texts_to_sequences(test_df.cutword)
## 将每个序列调整为相同的长度
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)

print(train_seq_mat.shape)
print(test_seq_mat.shape)

(782, 600)
(198, 600)


In [17]:
## 定义LSTM模型
inputs = Input(name='inputs',shape=[max_len])
## Embedding(词汇表大小,batch大小,每个新闻的词长)
layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
layer = LSTM(128)(layer)
layer = Dense(128,activation="relu",name="FC1")(layer)
layer = Dense(2,activation="softmax",name="FC2")(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(),metrics=["accuracy"])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 600)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 600, 128)          640128    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
FC1 (Dense)                  (None, 128)               16512     
_________________________________________________________________
FC2 (Dense)                  (None, 2)                 258       
Total params: 788,482
Trainable params: 788,482
Non-trainable params: 0
_________________________________________________________________


In [18]:
model_fit = model.fit(train_seq_mat,train_y,batch_size=128,epochs=10,
                      validation_data=(test_seq_mat,test_y)
                     )

Train on 782 samples, validate on 198 samples
Epoch 1/10
782/782 [==============================] - 8s 10ms/step - loss: 0.6847 - acc: 0.5614 - val_loss: 0.6586 - val_acc: 0.5707
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.6042 - acc: 0.6957 - val_loss: 0.6571 - val_acc: 0.6212
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4969 - acc: 0.8350 - val_loss: 0.5940 - val_acc: 0.6616
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3595 - acc: 0.8887 - val_loss: 0.7300 - val_acc: 0.6414
Epoch 5/10
782/782 [==============================] - 9s 11ms/step - loss: 0.2267 - acc: 0.9284 - val_loss: 0.6236 - val_acc: 0.6869
Epoch 6/10
782/782 [==============================] - 9s 12ms/step - loss: 0.1612 - acc: 0.9565 - val_loss: 0.6795 - val_acc: 0.7172
Epoch 7/10
782/782 [==============================] - 10s 12ms/step - loss: 0.1353 - acc: 0.9693 - val_loss: 0.7000 - val_acc: 0.6970
Epoch 8/10
782/782 [==

In [19]:
test_pre = model.predict(test_seq_mat)

## 评价预测效果，计算混淆矩阵
confm = metrics.confusion_matrix(np.argmax(test_pre,axis=1),np.argmax(test_y,axis=1))


In [20]:
confm

array([[66, 28],
       [18, 86]])